In [2]:
import cv2
import numpy as np
import math

In [5]:
cam_id = 1
api = cv2.CAP_DSHOW
res = (1280, 720)
capture = cv2.VideoCapture(cam_id, api)
focus = 0
capture.set(28, focus)
capture.set(3, 1280)
capture.set(4, 720)

False

In [1]:
291 * 23 / 30

223.1

In [ ]:
TILE_PX = 45
def calibrate(cam_img):
    cv2.circle(cam_img,(1280//2, 360), 5, (255, 255, 0), 2)
    cv2.line(cam_img, (1280//2, 0), (1280//2, 720), (0, 255, 0), 1)
    cv2.line(cam_img, (0, 360-TILE_PX//2), (1280, 360-TILE_PX//2), (255, 0, 0), 1)
    cv2.line(cam_img, (0, 360), (1280, 360), (255, 0, 0), 1)

def calc_circles(image):
    minr = 8
    maxr = 20
    edges = cv2.Canny(image, 100, 150)
    param1 = 200
    param2 = 30
    mindist = 10
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, mindist, param1=param1, param2=param2, maxRadius=maxr, minRadius=minr)
    edges2 = edges.copy()
    if circles is not None:
        for ind, i in enumerate(circles[0,:].astype(int)):
            cv2.circle(image,(i[0], i[1]), i[2], (0, 255, 0), 2)
            edges1 = edges.copy()
            #cv2.circle(edges1,(i[0], i[1]), i[2], (255, 255, 0), 2)
            #cv2.circle(edges2,(i[0], i[1]), i[2], (255, 255, 0), 1)
        #return edges2
        return i[:2]

In [4]:
np.tile

<function numpy.tile(A, reps)>

In [11]:
import numpy as np
img = np.full((100, 100, 4), 128).astype(np.uint8)
img[:, :, 0] = 0
img[:, :, 1] = 255
img[:, :, 2] = 0

In [14]:
aruco.DICT_4X4*?

aruco.DICT_4X4_100
aruco.DICT_4X4_1000
aruco.DICT_4X4_250
aruco.DICT_4X4_50

In [17]:
import cv2, cv2.aruco as aruco
img = cv2.imread('checker.png')


gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_1000)
parameters =  aruco.DetectorParameters_create()
corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

In [8]:

cv2.imshow('img', gray)
cv2.waitKey(10_000)


cv2.destroyAllWindows()

In [41]:
import numpy as np
import cv2
import glob
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((14*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:14].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Find the chess board corners
ret, corners = cv2.findChessboardCorners(gray, (7,14), None)
# If found, add object points, image points (after refining them)
if ret == True:
    objpoints.append(objp)
    corners2=cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
    imgpoints.append(corners)
    # Draw and display the corners
    img1 = img.copy()
    cv2.drawChessboardCorners(img1, (7,14), corners2, ret)
    cv2.imshow('img', img1)
    cv2.waitKey(10_000)

cv2.destroyAllWindows()

In [42]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [17]:
import cv2
while True:
    cv2.imshow('w', img)
    key = cv2.waitKey(1)
    img[:, :, 1] -= 1
    if key == ord('q'):
        break
        
cv2.destroyAllWindows()

In [6]:

num_of_samples = 10
ind = 0
avg = np.array([0, 0])
show_lines = False
i = 0
while True:
    _, cam_img = capture.read()
    
    if show_lines:
        calibrate(cam_img)
    
    #cv2.imshow('w', cam_img)
    
    imgray = cv2.cvtColor(cam_img, cv2.COLOR_BGR2GRAY)
    ret, edges = cv2.threshold(imgray, 150, 255, 0)
    #edges = cv2.Canny(cam_img, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = np.vectorize(cv2.contourArea)(contours)
    contours = np.array(contours, dtype=object)
    contours = contours[(areas > 500) & (areas < 2000)]
    
    rects = [cv2.minAreaRect(cnt) for cnt in contours]
    boxes = [np.int0(cv2.boxPoints(rect)) for rect in rects]
    
    good_boxes = []
    for box in boxes:
        M = cv2.moments(box)
        r = 360
        fig_center = np.array([int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])])
        center = np.array([640, 360-TILE_PX//2])
        if max(abs(fig_center-center)) < 210:
            if not i:
                print(fig_center)
            good_boxes.append(box)
            cv2.circle(cam_img, fig_center, 3, (255, 255, 0), -1)
    i = (i+1)%30
    cv2.drawContours(cam_img,contours,-1,(0,0,255),2)
    cv2.imshow('w', cam_img)
    key = cv2.waitKey(1)
    #print(key)
    if key == ord('q'):
        break
    if key == ord('c'):
        show_lines = not show_lines
cv2.destroyAllWindows()


[781 292]
[781 292]
[681 319]
[681 319]
[681 319]
[681 319]
